
<br>
=============================<br>
Straight line Hough transform<br>
=============================<br>
The Hough transform in its simplest form is a method to detect straight lines<br>
[1]_.<br>
In the following example, we construct an image with a line intersection. We<br>
then use the `Hough transform  <https://en.wikipedia.org/wiki/Hough_transform>`__.<br>
to explore a parameter space for straight lines that may run through the image.<br>
Algorithm overview<br>
------------------<br>
Usually, lines are parameterised as :math:`y = mx + c`, with a gradient<br>
:math:`m` and y-intercept `c`. However, this would mean that :math:`m` goes to<br>
infinity for vertical lines. Instead, we therefore construct a segment<br>
perpendicular to the line, leading to the origin. The line is represented by<br>
the length of that segment, :math:`r`, and the angle it makes with the x-axis,<br>
:math:`\\theta`.<br>
The Hough transform constructs a histogram array representing the parameter<br>
space (i.e., an :math:`M \\times N` matrix, for :math:`M` different values of<br>
the radius and :math:`N` different values of :math:`\\theta`).  For each<br>
parameter combination, :math:`r` and :math:`\\theta`, we then find the number<br>
of non-zero pixels in the input image that would fall close to the<br>
corresponding line, and increment the array at position :math:`(r, \\theta)`<br>
appropriately.<br>
We can think of each non-zero pixel "voting" for potential line candidates. The<br>
local maxima in the resulting histogram indicates the parameters of the most<br>
probable lines. In our example, the maxima occur at 45 and 135 degrees,<br>
corresponding to the normal vector angles of each line.<br>
Another approach is the Progressive Probabilistic Hough Transform [2]_. It is<br>
based on the assumption that using a random subset of voting points give a good<br>
approximation to the actual result, and that lines can be extracted during the<br>
voting process by walking along connected components. This returns the<br>
beginning and end of each line segment, which is useful.<br>
The function `probabilistic_hough` has three parameters: a general threshold<br>
that is applied to the Hough accumulator, a minimum line length and the line<br>
gap that influences line merging. In the example below, we find lines longer<br>
than 10 with a gap less than 3 pixels.<br>
References<br>
----------<br>
.. [1] Duda, R. O. and P. E. Hart, "Use of the Hough Transformation to<br>
       Detect Lines and Curves in Pictures," Comm. ACM, Vol. 15,<br>
       pp. 11-15 (January, 1972)<br>
.. [2] C. Galamhos, J. Matas and J. Kittler,"Progressive probabilistic<br>
       Hough transform for line detection", in IEEE Computer Society<br>
       Conference on Computer Vision and Pattern Recognition, 1999.<br>


####################<br>
Line Hough Transform<br>
====================

In [ ]:
import numpy as np

In [ ]:
from skimage.transform import hough_line, hough_line_peaks
from skimage.feature import canny
from skimage.draw import line
from skimage import data

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm

Constructing test image

In [ ]:
image = np.zeros((200, 200))
idx = np.arange(25, 175)
image[idx, idx] = 255
image[line(45, 25, 25, 175)] = 255
image[line(25, 135, 175, 155)] = 255

Classic straight-line Hough transform<br>
Set a precision of 0.5 degree.

In [ ]:
tested_angles = np.linspace(-np.pi / 2, np.pi / 2, 360, endpoint=False)
h, theta, d = hough_line(image, theta=tested_angles)

Generating figure 1

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 6))
ax = axes.ravel()

In [ ]:
ax[0].imshow(image, cmap=cm.gray)
ax[0].set_title('Input image')
ax[0].set_axis_off()

In [ ]:
angle_step = 0.5 * np.diff(theta).mean()
d_step = 0.5 * np.diff(d).mean()
bounds = [np.rad2deg(theta[0] - angle_step),
          np.rad2deg(theta[-1] + angle_step),
          d[-1] + d_step, d[0] - d_step]
ax[1].imshow(np.log(1 + h), extent=bounds, cmap=cm.gray, aspect=1 / 1.5)
ax[1].set_title('Hough transform')
ax[1].set_xlabel('Angles (degrees)')
ax[1].set_ylabel('Distance (pixels)')
ax[1].axis('image')

In [ ]:
ax[2].imshow(image, cmap=cm.gray)
ax[2].set_ylim((image.shape[0], 0))
ax[2].set_axis_off()
ax[2].set_title('Detected lines')

In [ ]:
for _, angle, dist in zip(*hough_line_peaks(h, theta, d)):
    (x0, y0) = dist * np.array([np.cos(angle), np.sin(angle)])
    ax[2].axline((x0, y0), slope=np.tan(angle + np.pi/2))

In [ ]:
plt.tight_layout()
plt.show()

#############################<br>
Probabilistic Hough Transform<br>
=============================

In [ ]:
from skimage.transform import probabilistic_hough_line

Line finding using the Probabilistic Hough Transform

In [ ]:
image = data.camera()
edges = canny(image, 2, 1, 25)
lines = probabilistic_hough_line(edges, threshold=10, line_length=5,
                                 line_gap=3)

Generating figure 2

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5), sharex=True, sharey=True)
ax = axes.ravel()

In [ ]:
ax[0].imshow(image, cmap=cm.gray)
ax[0].set_title('Input image')

In [ ]:
ax[1].imshow(edges, cmap=cm.gray)
ax[1].set_title('Canny edges')

In [ ]:
ax[2].imshow(edges * 0)
for line in lines:
    p0, p1 = line
    ax[2].plot((p0[0], p1[0]), (p0[1], p1[1]))
ax[2].set_xlim((0, image.shape[1]))
ax[2].set_ylim((image.shape[0], 0))
ax[2].set_title('Probabilistic Hough')

In [ ]:
for a in ax:
    a.set_axis_off()

In [ ]:
plt.tight_layout()
plt.show()